# Gramáticas Categoriales Combinatorias (GCC)

La entrega de laboratorio de GCC consiste en la resolución del presente *notebook*. 

A lo largo de este *notebook* encontrará bloques de código a completar y preguntas a responder. Se divide principalmente en dos partes, en la primera construirá las entradas léxicas y un parser de GCC utilizando NLTK para un conjunto de oraciones simples. En la segunda parte se consideran casos de mayor complejidad.

A lo largo de este *notebook*, usted:

- Interactuará con las representaciones para GCC de NLTK.
- Construirá las entradas léxicas necesarias para resolver los casos presentados.
- Construirá analizadores para GCC en NLTK y estudiará los resultados obtendidos.
- Se enfrentará a los principales problemas del enfoque y abordará algunos de ellos.


**Importante**: Ajústese en todo momento a las nociones linguísticas impartidas en el curso.


**Fecha de entrega**: Jueves 15 de junio.


### Pregunta

Comente brevemente en qué consiste el formalismo de GCC y enumere las reglas de derivación presentadas en el curso.

### Respuesta

Una gramatica categorial combinatoria propone el uso de operadores de lógica combinatoria ademas de los usados en la gramatica categorial. Es un formalismo de descripcion linguistica, el analisis linguistico se produce a partir de las definiciones de combinatoria sintagmatica especificadas en el lexico que luego se satisfacen por medio de reglas de cancelacion de argumentos sumamente abstractas

Componentes Básicos
Categorías: especifican tipos de palabras o constituyentes.
Reglas combinatorias: especifican cómo se pueden combinar las palabras o constituyentes. 
Léxico: especifica las categorías que puede tener una palabra.
Derivaciones: proceso sistemático de combinación de palabras o constituyentes.

Reglas
Aplicación funcional a la derecha
Aplicación funcional a la izquierda 
Elevación de tipo a la derecha
Elevación de tipo a la izquierda
Combinación a la derecha
Combinación a la izquierda
Coordinación

## Construcción de una GCC con NLTK
 
Ejecute el siguiente fragmento de código para importar módulos y ajustes iniciales que serán utilizados a lo largo de este notebook.

In [1]:
# Imports
from __future__ import print_function
from IPython.display import display

import nltk
from nltk.ccg import chart, lexicon
from nltk.ccg.chart import ApplicationRuleSet, CompositionRuleSet, TypeRaiseRuleSet

def display_derivation(p):
    chart.printCCGDerivation(p)


## Oraciones Simples

A continuación se construirá una GCC para el siguiente conjunto de oraciones.


In [2]:
oraciones_simples = [
    u"la ballena atraviesa el mar".split(),
    u"la ballena y el tiburón nadan".split(),
    u"la tortuga pone sus huevos en la costa".split(),
    u"la ballena atraviesa lentamente el mar".split(),
    u"lentamente las tortugas despiertan".split(),
    u"el tiburón blanco y azul nada con sus crías".split(),
]


En el bloque a continuación defina las entradas léxicas correspondientes.

Tenga en cuenta las siguientes consideraciones: 

- Utilice el conjunto de categorías simples: S, NP, N, PP.
- Considere las siguientes aridades para los verbos:
    - atravesar - 2 argumentos
    - nadar, despertar - 1 argumento
    - poner - 3 argumentos
- Considere una única categoría para la conjunción.
- No incluya las representaciones semánticas.

     

In [6]:
lex_simple_str = u'''
:- S, NP, N, PP
la => NP/N
las => NP/N
el => NP/N
sus => NP/N
ballena => N
tiburón => N
tortuga => N
tortugas => N
huevos => N
crías => N
mar => N
costa => N
atraviesa => (S\\NP)/NP
nadan => S\\NP
nada => S\\NP
pone => ((S\\NP)/PP)/NP
despiertan => S\\NP
y => var\\.,var/.,var
en => PP/NP
con => (S\\S)/NP
lentamente => S/S
lentamente => ((S\\NP)/NP)\\((S\\NP)/NP)
blanco => N\\N
azul => N\\N
'''

lex_simple = lexicon.fromstring(lex_simple_str)

Ejecute el bloque de código a continuación para construir el analizador y analizar las oraciones presentadas.

In [8]:
parser_simple = chart.CCGChartParser(lex_simple, ApplicationRuleSet)

Escriba en el siguiente bloque la rutina para desplegar todos los árboles de cada oración presentada. Utilice la función 'display_derivation' definida en el primer bloque.

In [9]:
# Despliegue de todos los árboles de cada oración

#
# Completar.
#
for sentence in oraciones_simples:
    print("Oracion: \t", ' '.join(sentence), "\n")
    for parse in parser_simple.parse(sentence):  
        display_derivation(parse)

Oracion: 	 la ballena atraviesa el mar 

   la    ballena   atraviesa     el    mar
 (NP/N)     N     ((S\NP)/NP)  (NP/N)   N
----------------->
       NP
                              ------------->
                                   NP
                 -------------------------->
                           (S\NP)
-------------------------------------------<
                     S
Oracion: 	 la ballena y el tiburón nadan 

   la    ballena              y                el    tiburón  nadan
 (NP/N)     N     ((_var0\.,_var0)/.,_var0)  (NP/N)     N     (S\NP)
----------------->
       NP
                                            ----------------->
                                                   NP
                 -------------------------------------------->
                                  (NP\.,NP)
-------------------------------------------------------------<
                             NP
---------------------------------------------------------------------<
                

### Pregunta

¿Este formalismo considera el problema de la concordancia? En caso afirmativo justifique, o proponga una extensión en caso contrario.

### Respuesta

El formalismo no considera el problema de concordancia, proponemos como extensión el agregar rasgos a los nombres, determinantes, adjetivos y verbos. 


proponemos el siguiente ejemplo para agregar los rasgos mencionados para la entrada "la gran belleza vive interiormente en uno"  y obtendriamos las siguientes diferencias:

en vez de
la => NP/N  (Determinante)
tener
la => NP[sg]/N[sg]

en vez de
gran => N/N   (Adjetivo-Modificador)
tener
gran=> N[sg]/N[sg]

en vez de
belleza => N
tener
belleza => N[sg]

en vez de
vive => (S\NP)/PP
tener
vive => (S\NP[sg])/PP

el adverbio queda igual
interiormente => (S/PP) \ (S/PP)

para el caso de concordancia en plurar el atributo o rasgo sería pl en lugar de sg.

## Oraciones más complejas

A continuación se extiende el conjunto de oraciones utilizado anteriormente.


In [10]:
oraciones_complejas = [
    u"la tortuga que el tiburón persiguió afortunadamente escapó".split(),
    u"la tortuga que nada lentamente atraviesa el mar".split(),
    
]

En el bloque a continuación extienda el léxico anteriormente construido para considerar las nuevas oraciones.

In [19]:
lex_str = lex_simple_str + u'''
nada => S[emb]\\NP
persiguió => S\\NP
persiguió => S[emb]\\NP
escapó => S\\NP
afortunadamente => (S\\NP)/(S\\NP)
afortunadamente => (S\\NP)\\(S\\NP)
afortunadamente => (S[emb]\\NP)/(S[emb]\\NP)
afortunadamente => (S[emb]\\NP)\\(S[emb]\\NP)
lentamente => ((S\\NP)/NP)/((S\\NP)/NP)
lentamente => ((S[emb]\\NP)/NP)\\((S[emb]\\NP)/NP)
lentamente => ((S[emb]\\NP)/NP)/((S[emb]\\NP)/NP)
que => (NP\\NP)/S[emb]
que => (NP\\NP)/(S[emb]\\NP)
'''

lex = lexicon.fromstring(lex_str)

Utilice el siguiente bloque de código para construir el parser para el léxico extendido y despliegue los árboles para todas las oraciones. 

Tenga en cuenta las siguientes consideraciones: 

- Utilice únicamente lo que ya ha sido importado.
- El código no debe superar las 10 lineas.


In [20]:
# Completar!
parser_complex = chart.CCGChartParser(lex, ApplicationRuleSet + CompositionRuleSet + TypeRaiseRuleSet)
for sentence in oraciones_complejas +  oraciones_simples:
    print("Oracion: \t", ' '.join(sentence), "\n")
    contador=1;
    for parse in parser_complex.parse(sentence):  
        print(contador);
        contador+=1;
        display_derivation(parse)

Oracion: 	 la tortuga que el tiburón persiguió afortunadamente escapó 

1
   la    tortuga         que            el    tiburón    persiguió    afortunadamente  escapó
 (NP/N)     N     ((NP\NP)/S['emb'])  (NP/N)     N     (S['emb']\NP)  ((S\NP)/(S\NP))  (S\NP)
----------------->
       NP
                                     ----------------->
                                            NP
                                     --------------------------------<
                                                 S['emb']
                 ---------------------------------------------------->
                                       (NP\NP)
                                                                     ------------------------->
                                                                              (S\NP)
                 -----------------------------------------------------------------------------<B
                                                    (S\NP)
----------------------

### Pregunta

¿Cuántas derivaciones obtuvo para cada una de las dos oraciones complejas? ¿Por qué ocurre esto? 

Indique para cada caso cuáles son correctas.

### Respuesta

### Pregunta

Explique la ambigüedad (no espúrea) que presenta la oración: "la tortuga que nada lentamente atraviesa el mar".

¿Es la única oración que presenta ambigüedad? Justifique.

### Respuesta

En "la tortuga que nada lentamente atraviesa el mar" existe ambigüedad no espúrea en la palabra lentamente, la cual es un adverbio y se encuentra entre dos verbos: nada y atraviesa; lentamente puede ser un modificador de un verbo tanto del lado izquierdo como derecho por lo que puede modificar tanto a nada (por derecha) como a atraviesa (por izquierda).

No es la única oración que presenta ambigüedad.
En la oración "la tortuga que el tiburón persiguió afortunadamente escapó" la palabra afortunadamente enfrenta el mismo problema que la palabra lentamente de la oración anterior.

Las oraciones simples no presentan ambigüedad no espúrea.


### Pregunta

¿Cambiaron las derivaciones de las oraciones simples? Justifique.

### Respuesta

Si, las derivaciones de las oraciones simples aumentaron esto es debido a que para el parser parser_simple solo se utilizaron las reglas de aplicación funcional. 
Al agregar las reglas de estas oraciones al parser parser_complex que si soporta reglas de derivación complejas las posibilidades de unificación de palabras y grupos de palabras aumentó generando nuevos árboles de derivaciones.
Esto hizo que se presentasen derivaciones incorrectas, por ejemplo: 
[la (NP/N) ]  [ballena N]   [atraviesa ((S\NP)/NP)]     [el (NP/N)]   [mar N]
la regla de combinación hace que se unan atraviesa y el para formar "atraviesa el"
